# **creating proportional codon and aa chnage dataframe** 

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np 
import matplotlib.patches as mpatches
import numpy as np 
import copy
import json
import pandas as pd
import math
from datetime import datetime

In [2]:
plt.style.use("seaborn-darkgrid")

In [3]:
#whole genome doubled model
model = json.load(open("../Human_mutability_model/Model_2020_12_02_genomeWide.txt"))
bases = [ "T", "C","A", "G"]

# **heat map using codons only** 
- going to need: 
    - DNAI dict
    - DNAF dict
    - DNA map dict... 
    - exon insertion dictionary 

In [4]:
file_path_afterSim = "data/dif_DNA/"
sim_name_afterSim = "Trial"
about_sim = "_cds_invariant50"
nTrials= 10 #
genToUse = 200000

In [5]:
DNA_dataDict = {}

#DifDNA800kgen trial 1-5
for index_tmp in range(1,nTrials+1):
    file_name_exondict = sim_name_afterSim+str(index_tmp)+"_exon_insertion_dict.txt"
    file_name_dnaI = sim_name_afterSim+str(index_tmp)+"_DNA_gen0_cds.txt"
    file_name_dnaF = sim_name_afterSim+str(index_tmp)+"_DNA_final"+about_sim+"_"+str(genToUse)+".txt"
    file_name_dnamap = sim_name_afterSim+str(index_tmp)+"_DNA_map"+about_sim+".txt"
    
    file_object_exondict = json.loads(open(file_path_afterSim+file_name_exondict).readlines()[0])
    file_object_dnaI = open(file_path_afterSim+file_name_dnaI).readlines()[0]
    file_object_dnaF = open(file_path_afterSim+file_name_dnaF).readlines()[0]
    file_object_dnamap = open(file_path_afterSim+file_name_dnamap).readlines()[0]
    
    DNA_dataDict["Trial"+str(index_tmp)] = [file_object_dnaI, file_object_dnaF, file_object_dnamap, file_object_exondict]  


#adding codonsI and codonsF list to the dna_data dictionary 
for trial_number in range(1,nTrials+1):
    trial_name = "Trial"+str(trial_number)

    current_DNAI = DNA_dataDict[trial_name][0]
    current_DNAF = DNA_dataDict[trial_name][1]
    current_codonsI = []
    current_codonsF = []

    tmp_nCodons = 0

    for exon_bounds_list in DNA_dataDict[trial_name][3].values():

        i = int(exon_bounds_list[0])+1#set the start of the codon (this is proven as the start)

        tmp_exon_codons = int((int(exon_bounds_list[1]) - int(exon_bounds_list[0]))/3)
        tmp_nCodons += tmp_exon_codons


        while i <= int(exon_bounds_list[1]): 
            current_codonsI.append(current_DNAI[i:i+3])
            current_codonsF.append(current_DNAF[i:i+3])
            i+= 3
        
    
    DNA_dataDict[trial_name].extend([current_codonsI, current_codonsF])

In [6]:
# creatign dictionary that has the initial, final, and (final-initial)/initial freq for each tripet (one list of each, n = ntrial) 
triplets = []
count_change_dictionary = {}
for base1 in bases: 
    for base2 in bases: 
        for base3 in bases: 
            
            triplet = base1+base2+base3
            triplets.append(triplet)
            count_change_dictionary[triplet] = [[],[],[]]
            
            for trial_number in range(1,nTrials+1):
                
                trial_name = "Trial"+str(trial_number)
                
                count_initial = DNA_dataDict[trial_name][4].count(triplet)
                count_final = DNA_dataDict[trial_name][5].count(triplet)
                
                count_change_dictionary[triplet][0].append(count_initial)
                count_change_dictionary[triplet][1].append(count_final)
                count_change_dictionary[triplet][2].append((count_final-count_initial)/count_initial)

In [7]:
codon_propChnage_dict = {}
for key, value_list in count_change_dictionary.items(): 
    codon_propChnage_dict[key] = value_list[2]

In [8]:
codon_propChange_df = pd.DataFrame.from_dict(codon_propChnage_dict)
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
codon_propChange_df.to_csv("data/dif_DNA/{t}_codon_propChnage_2xcov.csv".format(t=timestamp))

# **creating mean chnage for each a.a.** 

In [9]:
SynonymousCodons = {'CYS': ['TGT', 'TGC'], 'ASP': ['GAT', 'GAC'],
'SER': ['TCT', 'TCG', 'TCA', 'TCC', 'AGC', 'AGT'],
'GLN': ['CAA', 'CAG'], 'MET': ['ATG'], 'ASN': ['AAC', 'AAT'],
'PRO': ['CCT', 'CCG', 'CCA', 'CCC'], 'LYS': ['AAG', 'AAA'],
'STOP': ['TAG', 'TGA', 'TAA'], 'THR': ['ACC', 'ACA', 'ACG', 'ACT'],
'PHE': ['TTT', 'TTC'], 'ALA': ['GCA', 'GCC', 'GCG', 'GCT'],
'GLY': ['GGT', 'GGG', 'GGA', 'GGC'], 'ILE': ['ATC', 'ATA', 'ATT'],
'LEU': ['TTA', 'TTG', 'CTC', 'CTT', 'CTG', 'CTA'], 'HIS': ['CAT', 'CAC'],
'ARG': ['CGA', 'CGC', 'CGG', 'CGT', 'AGG', 'AGA'], 'TRP': ['TGG'],
'VAL': ['GTA', 'GTC', 'GTG', 'GTT'], 'GLU': ['GAG', 'GAA'], 'TYR': ['TAT', 'TAC']}

In [10]:
aa_mean_change = {}

for aa, codon_list in SynonymousCodons.items(): 
    aa_current_chnage_list = []
    for trial_number in range(nTrials): 
        sum_i = 0
        sum_f = 0
        for syn_codon in codon_list: 
            
            sum_i += count_change_dictionary[syn_codon][0][trial_number]
            sum_f += count_change_dictionary[syn_codon][1][trial_number]
        aa_current_chnage_list.append((sum_f-sum_i)/sum_i)
    aa_mean_change[aa] = aa_current_chnage_list

In [11]:
aa_mean_change_df = pd.DataFrame.from_dict(aa_mean_change)
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
aa_mean_change_df.to_csv("data/dif_DNA/{t}_aa_propChnage_2xcov.csv".format(t=timestamp))

## **create df for R regression analysis mutability on codon chnage**

In [23]:
r_codon_change = {}

for codon,value in codon_propChnage_dict.items():
    r_codon_change[codon] = [model[codon][0], np.log10(model[codon][0]),np.mean(value)]

In [25]:
r_codon_change_df = pd.DataFrame.from_dict(r_codon_change,orient="index",columns=["mut","mut_log10","codon_chnage_2x"])

In [26]:
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
r_codon_change_df.to_csv("data/dif_DNA/{t}_codon_propChnage_2xcov_Rdf.csv".format(t=timestamp))